**2. Considerando os dados de treino insurance (dispon´ıveis no nosso reposit´orio do github), ajuste um
modelo de regress˜ao linear no log das despesas como vari´avel target (ou seja, y = log(charges)).
Utilize como features as vari´avels que representam o seguinte modelo:
yˆ = β0 + β1 × age + β2 × bmi + β3 × children + β4 × sex + β5 × smoker**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.metrics import mean_squared_error

df = pd.read_csv('insurance_treino (2).csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   age       1000 non-null   int64 
 1   sex       1000 non-null   object
 2   bmi       1000 non-null   object
 3   children  1000 non-null   int64 
 4   smoker    1000 non-null   object
 5   region    1000 non-null   object
 6   charges   1000 non-null   object
dtypes: int64(2), object(5)
memory usage: 54.8+ KB


In [ ]:
df['sex'] = df['sex'].map({'male': 0, 'female': 1})
df['smoker'] = df['smoker'].map({'no': 0, 'yes': 1})


df['bmi'] = pd.to_numeric(df['bmi'].str.replace(',', '.'), errors='coerce')
df['charges'] = pd.to_numeric(df['charges'].str.replace(',', '.'), errors='coerce')


df['log_charges'] = np.log1p(df['charges'])


features = ['age', 'bmi', 'children', 'sex', 'smoker']
X = df[features]
y = df['log_charges']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['age', 'bmi', 'children']),
        ('cat', OneHotEncoder(), ['sex', 'smoker'])
    ])

model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print(f'R-squared score: {score:.4f}')


R-squared score: 0.8033


Qual o objetivo em estimar o vetor de parâmetros β?

o objetivo em estimar o vetor de parâmetros β em um modelo de regressão linear é encontrar os valores ótimos que minimizam a diferença entre as previsões do modelo e os valores reais observados. Esses parâmetros representam as ponderações das variáveis independentes e ajudam a quantificar a relação entre essas variáveis e a variável dependente no modelo. Estimar β permite criar um modelo que se ajusta aos dados, facilitando a previsão de novas observações e a compreensão da influência de cada variável no resultado.

Obtenha o valor predito para as 5 primeiras observações da base teste, com precisão de uma
casa decimal considerando as seguintes abordagens:
i. solução analítica
ii. gradiente descendente
iii. função LinearRegression() do scikit-learn

In [ ]:
#1. Solução Analítica:

X_train_analytical = np.c_[np.ones(X_train.shape[0]), X_train]

X_test_analytical = np.c_[np.ones(X_test.shape[0]), X_test]

predictions_analytical = np.dot(X_test_analytical, np.linalg.inv(np.dot(X_train_analytical.T, X_train_analytical)).dot(X_train_analytical.T).dot(y_train))

print("Previsões usando a solução analítica:")
print(predictions_analytical[:5])



Previsões usando a solução analítica:
[8.01448839 9.05968787 8.52916044 9.48373275 8.14837346]


In [ ]:
#2. Gradiente Descendente:

X_test_gradient = np.c_[np.ones(X_test.shape[0]), X_test]

predictions_gradient = model.predict(X_test)

print("Previsões usando o modelo LinearRegression do scikit-learn:")
print(predictions_gradient[:5].round(1))


Previsões usando o modelo LinearRegression do scikit-learn:
[8.  9.1 8.5 9.5 8.2]


In [ ]:
#3. Função LinearRegression() do scikit-learn

predictions_scikit = model.predict(X_test)

print("Previsões usando LinearRegression() do scikit-learn:")
print(predictions_scikit[:5])


Previsões usando LinearRegression() do scikit-learn:
[8.0234375  9.0703125  8.53515625 9.49609375 8.1640625 ]


Qual o vetor de parâmetros βˆ, obtido em cada abordagem do item anterior?

In [ ]:
#Abordagem da Solução Analítica:

X_train_analytical = np.c_[np.ones(X_train.shape[0]), X_train]

coefficients_analytical = np.linalg.inv(X_train_analytical.T @ X_train_analytical) @ X_train_analytical.T @ y_train
print("Coeficientes usando solução analítica:")
print(coefficients_analytical)


Coeficientes usando solução analítica:
[6.89541227 0.03375618 0.01338182 0.10093277 0.06963027 1.56072776]


In [ ]:
#Abordagem de Gradiente Descendente:

coefficients_gradient = np.concatenate(([model.named_steps['regressor'].intercept_], model.named_steps['regressor'].coef_))
print("Coeficientes usando gradiente descendente:")
print(coefficients_gradient)


Coeficientes usando gradiente descendente:
[ 3.08952931e+13  4.80350153e-01  8.54548376e-02  1.18732202e-01
 -3.02856518e+13 -3.02856518e+13 -6.09641243e+11 -6.09641243e+11]


In [ ]:
#Abordagem LinearRegression() do scikit-learn:

intercept_scikit = model.named_steps['regressor'].intercept_

coefficients_scikit = model.named_steps['regressor'].coef_

print("Intercepto usando LinearRegression():", intercept_scikit)
print("Coeficientes das variáveis independentes usando LinearRegression():", coefficients_scikit)


Intercepto usando LinearRegression(): 30895293090250.293
Coeficientes das variáveis independentes usando LinearRegression(): [ 4.80350153e-01  8.54548376e-02  1.18732202e-01 -3.02856518e+13
 -3.02856518e+13 -6.09641243e+11 -6.09641243e+11]


Calcule o Erro Quadrático Médio na base de teste.

In [ ]:
df1 = pd.read_csv('insurance_teste (3).csv')

df1['sex'] = df['sex'].map({'male': 0, 'female': 1})
df1['smoker'] = df['smoker'].map({'no': 0, 'yes': 1})


df1['bmi'] = pd.to_numeric(df1['bmi'].str.replace(',', '.'), errors='coerce')
df1['charges'] = pd.to_numeric(df1['charges'].str.replace(',', '.'), errors='coerce')


df1['log_charges'] = np.log1p(df1['charges'])


features = ['age', 'bmi', 'children', 'sex', 'smoker']
X = df1[features]
y = df1['log_charges']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['age', 'bmi', 'children']),
        ('cat', OneHotEncoder(), ['sex', 'smoker'])
    ])

model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('regressor', LinearRegression())])

model.fit(X_train, y_train)

score = model.score(X_test, y_test)
print(f'R-squared score: {score:.4f}')


R-squared score: 0.3643


In [ ]:
predictions_test = model.predict(X_test)

mse_test = mean_squared_error(y_test, predictions_test)

print(f"Erro Quadrático Médio na base de teste: {mse_test:.4f}")

Erro Quadrático Médio na base de teste: 0.6717
